In [185]:
import math
import pandas as pd

In [2]:
in_files = [
    "a_example.txt"
    , "b_read_on.txt"
    , "c_incunabula.txt"
    , "d_tough_choices.txt"
    , "e_so_many_books.txt"
    , "f_libraries_of_the_world.txt"
]

In [3]:
def to_int_list(line):
    return [b for b in map(int, line.strip().split(" "))]

def read_input(in_file):
    with open(in_file, 'r') as f:
        l = f.readline().strip()
        first_data = [d for d in map(int, l.split(" "))]
        B = first_data[0]
        L = first_data[1]
        D = first_data[2]
        print(f'Total Books: {B}\nLibraries: {L}\nDays: {D}')
        
        book_scores = [b for b in map(int, f.readline().strip().split(" "))]

#        print(book_scores)
        libraries = []
        for idx, library in enumerate(range(L)):
            lib_data_1 = to_int_list(f.readline())
            number_of_books = lib_data_1[0]
            time = lib_data_1[1]
            b_per_day = lib_data_1[2]
#            print(f'{library} -> books: {number_of_books}, Time: {time}, BxDay: {b_per_day}')
            lib_books = to_int_list(f.readline())
#            print(f'{library} -> Books: {lib_books}')
            libraries.append({"Library": idx
                              , "Number of books": number_of_books
                              , "Startup time": time
                              , "Books per day": b_per_day
                              , "Books": lib_books
                             })
        
        return ({
            "Total Books": B
            , "Total Libraries": L
            , "Total Days": D
            , "Books Scores": book_scores
            , "Libraries": libraries
        })


In [4]:
foo = read_input("a_example.txt")
bar = pd.DataFrame(foo['Libraries'])

Total Books: 6
Libraries: 2
Days: 7


In [198]:
books = pd.DataFrame(foo['Books Scores'])

In [107]:
sorted_books = books.sort_values(by=0, ascending=False)

In [116]:
libraries = pd.DataFrame(foo['Libraries'])

In [119]:
libraries.sort_values('Startup time', inplace=True)

In [120]:
libraries

,Library,Number of books,Startup time,Books per day,Books
0,0,5,2,2,"[0, 1, 2, 3, 4]"
1,1,4,3,1,"[0, 2, 3, 5]"


In [123]:
signed_libraries = libraries.shape[0]

In [124]:
signed_libraries

2

In [171]:
for idx, library in libraries.iterrows():
    first_line = f"{library['Library']} {len(library['Books'])}"
    second_line = " ".join([str(b) for b in library['Books']])
    print(f'{first_line}\n{second_line}')

0 5
0 1 2 3 4
1 4
0 2 3 5


In [159]:
def write_out_file(out_file, solution):
    with open(out_file, 'w') as o:
        o.write(f"{solution.shape[0]}\n")
        for idx, library in solution.iterrows():
            first_line = f"{library['Library']} {len(library['Books'])}\n"
            second_line = " ".join([str(b) for b in library['Books']]) + "\n"
#            print(f'{first_line}\n{second_line}')
            o.write(first_line)
            o.write(second_line)
    

In [179]:
write_out_file("foo.out", libraries)

0 5

0 1 2 3 4

1 4

0 2 3 5



In [332]:
def solve(solution):
    for problem in in_files:
        print(problem)
        data_in = read_input(problem)
        libraries = pd.DataFrame(data_in['Libraries'])
        solution = solution(problem)
        write_out_file(problem.replace(".txt", ".out"), solution)
        print("\n\n")

# Naive

In [197]:
for problem in in_files[-1:]:
    print(problem)
    data_in = read_input(problem)
    libraries = pd.DataFrame(data_in['Libraries'])
    libraries.sort_values('Startup time', inplace=True)
    write_out_file(problem.replace(".txt", ".out"), libraries)
    print("\n\n")

f_libraries_of_the_world.txt
Total Books: 100000
Libraries: 1000
Days: 700





In [184]:
"foo.txt".replace(".txt", ".out")

'foo.out'

# More naive

In [326]:
def find_solution(in_file):
    problem_data = read_input(in_file)
    books = pd.DataFrame(problem_data['Books Scores'])
    libraries = pd.DataFrame(problem_data['Libraries'])
    libraries.sort_values('Startup time', inplace=True)
    solution = libraries
    for idx , library in libraries.iterrows():
        library_books = pd.DataFrame(library['Books']).set_index(0)
        scan_books = books.index.intersection(library_books.index)
        if len(scan_books) == 0:
            solution.drop(index=[library['Library']], inplace=True)
        else:
            books.drop(index=scan_books, inplace=True)
            solution.at[library['Library'], 'Books'] = list(scan_books)
        
    return solution
    


In [327]:
foo = find_solution("a_example.txt")

Total Books: 6
Libraries: 2
Days: 7


In [328]:
foo.head()

,Library,Number of books,Startup time,Books per day,Books
0,0,5,2,2,"[0, 1, 2, 3, 4]"
1,1,4,3,1,[5]


In [331]:
for problem in in_files:
    print(problem)
    data_in = read_input(problem)
    libraries = pd.DataFrame(data_in['Libraries'])
    solution = find_solution(problem)
    write_out_file(problem.replace(".txt", ".out"), solution)
    print("\n\n")

a_example.txt
Total Books: 6
Libraries: 2
Days: 7
Total Books: 6
Libraries: 2
Days: 7



b_read_on.txt
Total Books: 100000
Libraries: 100
Days: 1000
Total Books: 100000
Libraries: 100
Days: 1000



c_incunabula.txt
Total Books: 100000
Libraries: 10000
Days: 100000
Total Books: 100000
Libraries: 10000
Days: 100000



d_tough_choices.txt
Total Books: 78600
Libraries: 30000
Days: 30001
Total Books: 78600
Libraries: 30000
Days: 30001



e_so_many_books.txt
Total Books: 100000
Libraries: 1000
Days: 200
Total Books: 100000
Libraries: 1000
Days: 200



f_libraries_of_the_world.txt
Total Books: 100000
Libraries: 1000
Days: 700
Total Books: 100000
Libraries: 1000
Days: 700





# super naive

In [383]:
def super_naive_solution(in_file):
    problem_data = read_input(in_file)
    libri = problem_data['Books Scores']
    for i in problem_data['Libraries']:
        i['points']=sum([libri[j] for j in i['Books']])
    books = pd.DataFrame(problem_data['Books Scores'])
    libraries = pd.DataFrame(problem_data['Libraries'])
    libraries['Value'] = (problem_data['Total Days'] - libraries['Startup time']) * libraries['Books per day'] + libraries['points']
#    print(libraries['Value'])
    libraries.sort_values('Value', inplace=True, ascending=False)
    solution = libraries
    for idx , library in libraries.iterrows():
        library_books = pd.DataFrame(library['Books']).set_index(0)
        scan_books = books.index.intersection(library_books.index)
        if len(scan_books) == 0:
            solution.drop(index=[library['Library']], inplace=True)
        else:
            books.drop(index=scan_books, inplace=True)
            solution.at[library['Library'], 'Books'] = list(scan_books)
        
    return solution

In [384]:
foo = super_naive_solution("a_example.txt")

Total Books: 6
Libraries: 2
Days: 7


In [388]:
for problem in in_files[3:]:
    print(problem)
    data_in = read_input(problem)
    libraries = pd.DataFrame(data_in['Libraries'])
    solution = super_naive_solution(problem)
    write_out_file(problem.replace(".txt", ".out"), solution)
    print("\n\n")

d_tough_choices.txt
Total Books: 78600
Libraries: 30000
Days: 30001
Total Books: 78600
Libraries: 30000
Days: 30001



e_so_many_books.txt
Total Books: 100000
Libraries: 1000
Days: 200
Total Books: 100000
Libraries: 1000
Days: 200



f_libraries_of_the_world.txt
Total Books: 100000
Libraries: 1000
Days: 700
Total Books: 100000
Libraries: 1000
Days: 700





# Naive as much as I can

In [220]:
def iper_naive_solution(in_file):
    problem_data = read_input(in_file)
    books = pd.DataFrame(problem_data['Books Scores'])
    libraries = pd.DataFrame(problem_data['Libraries'])
    libraries['Books'] = libraries['Books'].apply(lambda b: sorted(b, reverse=True))
    days_left = problem_data['Total Days']
#    libraries['Value'] = (problem_data['Total Days'] - libraries['Startup time']) * libraries['Books per day'] + libraries['points']
    
    solution = None
    while days_left > 0 and libraries.shape[0] > 0 and books.shape[0] > 0:
        def library_value(library):
            possible_books = books.loc[library['Books']]
#            print(possible_books.shape)
            available_days = days_left - library['Startup time'] - 1
            if possible_books.shape[0] == 0 or available_days <= 0:
                return 0
            points = books.loc[library['Books'][:available_days]].sum()
            production_days = min(
                math.ceil(possible_books.shape[0] / library['Books per day'])
                , available_days
            )
            
#            return (points / library['Books per day']) * (days_left - library['Startup time'])
            return points #(days_left - library['Startup time'])
        
        libraries['Value'] = libraries.apply(library_value, axis=1)
        libraries = libraries[libraries['Value'] > 0]
        if libraries.shape[0] == 0:
            break
        libraries = libraries.sort_values('Value', ascending=False)
        the_library = libraries.iloc[0]
        days_left = days_left - the_library['Startup time']
        print(days_left)
        if solution is None:
            solution = libraries.iloc[[0]]
        else:
            solution = solution.append(libraries.iloc[[0]])
        libraries = libraries.drop(index=the_library['Library'])
        library_books = pd.DataFrame(the_library['Books']).set_index(0)
        scan_books = books.index.intersection(library_books.index)
        books = books.drop(index=scan_books)
        libraries['Books'] = libraries['Books'].apply(lambda b: list(books.index.intersection(b)))
#        print(the_library)


        
    return solution

In [221]:
#the_solution = iper_naive_solution("b_read_on.txt")

In [225]:
for problem in in_files[0:2]:
    print(problem)
    data_in = read_input(problem)
    libraries = pd.DataFrame(data_in['Libraries'])
    solution = iper_naive_solution(problem)
    write_out_file(problem.replace(".txt", ".out"), solution)
    print("\n\n")

a_example.txt
Total Books: 6
Libraries: 2
Days: 7
Total Books: 6
Libraries: 2
Days: 7
5
2



b_read_on.txt
Total Books: 100000
Libraries: 100
Days: 1000
Total Books: 100000
Libraries: 100
Days: 1000
999
998
996
994
992
990
988
985
982
979
976
972
968
964
960
956
951
946
941
936
930
924
918
912
905
898
891
884
876
868
860
852
844
836
828
820
812
803
793
783
773
763
753
742
731
719
707
695
683
671
658
645
631
617
603
589
574
559
544
529
514
499
484
469
454
439
423
407
391
375
359
343
326
309
292
275
258
241
223
205
187
169
151
133
115
97
78
59
39
19





In [7]:
foo = read_input("a_example.txt")
bar = pd.DataFrame(foo['Libraries'])

Total Books: 6
Libraries: 2
Days: 7


In [9]:
books = pd.DataFrame(foo['Books Scores'])
sorted_books = books.sort_values(by=0, ascending=False)

In [143]:
books.drop([0])

,0
1,2
2,3
3,6
4,5
5,4


In [13]:
books

,0
0,1
1,2
2,3
3,6
4,5
5,4


In [117]:
libraries = pd.DataFrame(foo['Libraries'])

In [23]:
libraries.apply(lambda r: books.loc[r['Books']].sum(), axis=1)

,0
0,17
1,14


In [24]:
print(foo)

{'Total Books': 6, 'Total Libraries': 2, 'Total Days': 7, 'Books Scores': [1, 2, 3, 6, 5, 4], 'Libraries': [{'Library': 0, 'Number of books': 5, 'Startup time': 2, 'Books per day': 2, 'Books': [0, 1, 2, 3, 4]}, {'Library': 1, 'Number of books': 4, 'Startup time': 3, 'Books per day': 1, 'Books': [0, 2, 3, 5]}]}


In [206]:
[1, 2, 4, 5][:2]

[1, 2]